# Projeto 1 - Ciência dos Dados

Nome: Gabriel Tkacz

Nome: Guilherme Carvalho

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [168]:
!pip install emoji
!pip install emoji_list
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display
pd.options.display.max_rows = 13

In [169]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\guilh\Desktop\projeto1cdados


In [170]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed1 = re.sub(r"http\S+", "", text)
    text_subbed = re.sub(pattern, ' ', text_subbed1)
    ''''acentosa=["á","à","ã","â"]
    acentose=["é","è","ê"]
    acentoso=["ó","ò","ô","õ"]
    acentosu=["û","ú","ù",]
    acentosc=["ç"]
    for i in range(len(text_subbed)):
        if text_subbed[i] in acentosa:
            text_subbed[i]='a'
        if text_subbed[i] in acentose:
            text_subbed[i]='e'
        if text_subbed[i] in acentoso:
            text_subbed[i]='o'
        if text_subbed[i] in acentosu:
            text_subbed[i]='u'
        if text_subbed[i] in acentosc:
            text_subbed[i]='c'
    for i in range(len(text_subbed)):
        if text_subbed[i]=='@':
            for z in range(len(text_subbed[i:])):
                if text_subbed[i:][z]==" " or text_subbed[i:][z]==":":
                    del text_subbed[i:z]
                    break'''
    return text_subbed

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [171]:
dados = pd.read_excel('NuBank.xlsx')
dados

,Treinamento,classificação
0,@jaum_lealf @cecbreno @nubank amigo que triste...,0
1,"@oandrevinic aaaah, que notícia boa!\n\nconta ...",0
2,quando o nubank vai liberar a minha função cré...,0
3,@catundabruno @nubank eu vou cancelar. não hou...,0
4,"nubank aumentou meu limite, to fudida 🥺😭😂",0
...,...,...
295,se tem uma coisa que eu amo é meu cartão nuban...,0
296,rt @wjucollection: 🌱 itzy\n\nalbum it’z icy: f...,0
297,aff meu nubank novo é mt lindo,0
298,"olhei o dólar canadense hoje e tá r$ 3,39 e ta...",0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [172]:
#definindo variavel categorica
dados.loc[:,'classificação'] = dados['classificação'].astype('category')
dados['classificação'].cat.categories = ["irrelevante", "relevante"]
dados

,Treinamento,classificação
0,@jaum_lealf @cecbreno @nubank amigo que triste...,irrelevante
1,"@oandrevinic aaaah, que notícia boa!\n\nconta ...",irrelevante
2,quando o nubank vai liberar a minha função cré...,irrelevante
3,@catundabruno @nubank eu vou cancelar. não hou...,irrelevante
4,"nubank aumentou meu limite, to fudida 🥺😭😂",irrelevante
...,...,...
295,se tem uma coisa que eu amo é meu cartão nuban...,irrelevante
296,rt @wjucollection: 🌱 itzy\n\nalbum it’z icy: f...,irrelevante
297,aff meu nubank novo é mt lindo,irrelevante
298,"olhei o dólar canadense hoje e tá r$ 3,39 e ta...",irrelevante


In [173]:
#tweets relevantes:
rel = dados["classificação"] == "relevante"
relevantes = dados.loc[rel,["Treinamento"]].copy()
relevantes

,Treinamento
7,"@rodrigo_rhr @nubank @bradesco aff, pistola es..."
8,"app da digo é uma bosta, mil vezes nubank"
11,@newspotgabs @nubank so queria que me dessem l...
12,@vitorswu @nubank @bradesco nossa direto os ca...
14,chegou o meu cartão da nubank aaaaa 😍😍
...,...
251,"@nubank podia aumentar meu limite, não está da..."
258,"@minimesq sim, e ai?? vai aparecer quando?\nre..."
265,nubank ja pode libera meu cartão
289,@nubank poderia rolar um aumento de comemoraçã...


In [174]:
#tweets irrelevantes:
irel = dados["classificação"] == "irrrelevante"
irrelevantes = dados.loc[rel,["Treinamento"]].copy()
irrelevantes

,Treinamento
7,"@rodrigo_rhr @nubank @bradesco aff, pistola es..."
8,"app da digo é uma bosta, mil vezes nubank"
11,@newspotgabs @nubank so queria que me dessem l...
12,@vitorswu @nubank @bradesco nossa direto os ca...
14,chegou o meu cartão da nubank aaaaa 😍😍
...,...
251,"@nubank podia aumentar meu limite, não está da..."
258,"@minimesq sim, e ai?? vai aparecer quando?\nre..."
265,nubank ja pode libera meu cartão
289,@nubank poderia rolar um aumento de comemoraçã...


In [175]:
lista_relevante= relevantes["Treinamento"].tolist()
lista_relevante

['@rodrigo_rhr @nubank @bradesco aff, pistola estou kkk se eu estivesse devendo ou com score baixo até entenderia.',
 'app da digo é uma bosta, mil vezes nubank',
 '@newspotgabs @nubank so queria que me dessem limite',
 '@vitorswu @nubank @bradesco nossa direto os caras aumentam meu limite kk e olha q eu nem trabalho',
 'chegou o meu cartão da nubank aaaaa 😍😍',
 'até hj o novo cartão não chegou. quando fui perguntar a atendente da @nubank disse q não tinha nada disso registrado e q eu tinha cancelado o cartão pq não queria mais.',
 '@malualcure1 @nubank taxa 6,50 no saque pra quebrar a cara e por a frase em efeito (não existe nada perfeito nesse mundo)',
 'eu que lute pra krl pra pagar essa porra, mdssssss quero meu nubank de volta https://t.co/viz1jeeeea',
 'momento raro pra mim: vou elogiar algo.\nprbns pro sistema do @nubank que conseguiu detectar uma fraude no meu cartão de crédito virtual, bloquear a transação e o cartão e me avisar imediatamente. :)\nñ é fácil definir meu padrão 

In [176]:
lista_irrelevante= irrelevantes["Treinamento"].tolist()


In [177]:
#criandotxt
lista_relevante2 = ''.join(lista_relevante)
lista_relevante2

'@rodrigo_rhr @nubank @bradesco aff, pistola estou kkk se eu estivesse devendo ou com score baixo até entenderia.app da digo é uma bosta, mil vezes nubank@newspotgabs @nubank so queria que me dessem limite@vitorswu @nubank @bradesco nossa direto os caras aumentam meu limite kk e olha q eu nem trabalhochegou o meu cartão da nubank aaaaa 😍😍até hj o novo cartão não chegou. quando fui perguntar a atendente da @nubank disse q não tinha nada disso registrado e q eu tinha cancelado o cartão pq não queria mais.@malualcure1 @nubank taxa 6,50 no saque pra quebrar a cara e por a frase em efeito (não existe nada perfeito nesse mundo)eu que lute pra krl pra pagar essa porra, mdssssss quero meu nubank de volta https://t.co/viz1jeeeeamomento raro pra mim: vou elogiar algo.\nprbns pro sistema do @nubank que conseguiu detectar uma fraude no meu cartão de crédito virtual, bloquear a transação e o cartão e me avisar imediatamente. :)\nñ é fácil definir meu padrão de compras, ainda mais com 3 pessoas util

In [178]:
lista_irrelevante2 = ''.join(lista_irrelevante)
lista_irrelevante2

'@rodrigo_rhr @nubank @bradesco aff, pistola estou kkk se eu estivesse devendo ou com score baixo até entenderia.app da digo é uma bosta, mil vezes nubank@newspotgabs @nubank so queria que me dessem limite@vitorswu @nubank @bradesco nossa direto os caras aumentam meu limite kk e olha q eu nem trabalhochegou o meu cartão da nubank aaaaa 😍😍até hj o novo cartão não chegou. quando fui perguntar a atendente da @nubank disse q não tinha nada disso registrado e q eu tinha cancelado o cartão pq não queria mais.@malualcure1 @nubank taxa 6,50 no saque pra quebrar a cara e por a frase em efeito (não existe nada perfeito nesse mundo)eu que lute pra krl pra pagar essa porra, mdssssss quero meu nubank de volta https://t.co/viz1jeeeeamomento raro pra mim: vou elogiar algo.\nprbns pro sistema do @nubank que conseguiu detectar uma fraude no meu cartão de crédito virtual, bloquear a transação e o cartão e me avisar imediatamente. :)\nñ é fácil definir meu padrão de compras, ainda mais com 3 pessoas util

In [179]:
#ceiando arquivo texto relevantes:
arquivo = open("arqR.txt","w",encoding="utf-8-sig")
arquivo.write(lista_relevante2)
arquivo.close()

In [180]:
#criando arquivo texto irrelevantes:
arquivo = open("arqIR.txt","w",encoding="utf-8-sig")
arquivo.write(lista_irrelevante2)
arquivo.close()

In [181]:
#lendo arquivo texto relevantes:
with open("arqR.txt", "r", encoding="utf-8-sig") as arquivo_texto:
    texto_relevante = arquivo_texto.read()

# Imprime os 100 primeiros caracteres do texto para verificar se está tudo ok.
print(texto_relevante[0:404])

@rodrigo_rhr @nubank @bradesco aff, pistola estou kkk se eu estivesse devendo ou com score baixo até entenderia.app da digo é uma bosta, mil vezes nubank@newspotgabs @nubank so queria que me dessem limite@vitorswu @nubank @bradesco nossa direto os caras aumentam meu limite kk e olha q eu nem trabalhochegou o meu cartão da nubank aaaaa 😍😍até hj o novo cartão não chegou. quando fui perguntar a atendente


In [182]:
#lendo arquivo texto irrelevantes:
with open("arqIR.txt", "r", encoding="utf-8-sig") as arquivo_texto:
    texto_irrelevante = arquivo_texto.read()

# Imprime os 100 primeiros caracteres do texto para verificar se está tudo ok.
print(texto_relevante[0:404])

@rodrigo_rhr @nubank @bradesco aff, pistola estou kkk se eu estivesse devendo ou com score baixo até entenderia.app da digo é uma bosta, mil vezes nubank@newspotgabs @nubank so queria que me dessem limite@vitorswu @nubank @bradesco nossa direto os caras aumentam meu limite kk e olha q eu nem trabalhochegou o meu cartão da nubank aaaaa 😍😍até hj o novo cartão não chegou. quando fui perguntar a atendente


In [183]:
texto_R_clean = cleanup(texto_relevante.lower())
texto_R_clean

'@rodrigo_rhr @nubank @bradesco aff  pistola estou kkk se eu estivesse devendo ou com score baixo até entenderia app da digo é uma bosta  mil vezes nubank@newspotgabs @nubank so queria que me dessem limite@vitorswu @nubank @bradesco nossa direto os caras aumentam meu limite kk e olha q eu nem trabalhochegou o meu cartão da nubank aaaaa 😍😍até hj o novo cartão não chegou  quando fui perguntar a atendente da @nubank disse q não tinha nada disso registrado e q eu tinha cancelado o cartão pq não queria mais @malualcure1 @nubank taxa 6 50 no saque pra quebrar a cara e por a frase em efeito  não existe nada perfeito nesse mundo eu que lute pra krl pra pagar essa porra  mdssssss quero meu nubank de volta  raro pra mim  vou elogiar algo \nprbns pro sistema do @nubank que conseguiu detectar uma fraude no meu cartão de crédito virtual  bloquear a transação e o cartão e me avisar imediatamente    \nñ é fácil definir meu padrão de compras  ainda mais com 3 pessoas utilizando   mais um mês pagando o

In [184]:
texto_IR_clean = cleanup(texto_irrelevante.lower())
texto_IR_clean

'@rodrigo_rhr @nubank @bradesco aff  pistola estou kkk se eu estivesse devendo ou com score baixo até entenderia app da digo é uma bosta  mil vezes nubank@newspotgabs @nubank so queria que me dessem limite@vitorswu @nubank @bradesco nossa direto os caras aumentam meu limite kk e olha q eu nem trabalhochegou o meu cartão da nubank aaaaa 😍😍até hj o novo cartão não chegou  quando fui perguntar a atendente da @nubank disse q não tinha nada disso registrado e q eu tinha cancelado o cartão pq não queria mais @malualcure1 @nubank taxa 6 50 no saque pra quebrar a cara e por a frase em efeito  não existe nada perfeito nesse mundo eu que lute pra krl pra pagar essa porra  mdssssss quero meu nubank de volta  raro pra mim  vou elogiar algo \nprbns pro sistema do @nubank que conseguiu detectar uma fraude no meu cartão de crédito virtual  bloquear a transação e o cartão e me avisar imediatamente    \nñ é fácil definir meu padrão de compras  ainda mais com 3 pessoas utilizando   mais um mês pagando o

In [185]:
texto_total = texto_R_clean + texto_IR_clean

In [186]:
totdas_palavras_relevantes = texto_R_clean.split()
totdas_palavras_relevantes

['@rodrigo_rhr',
 '@nubank',
 '@bradesco',
 'aff',
 'pistola',
 'estou',
 'kkk',
 'se',
 'eu',
 'estivesse',
 'devendo',
 'ou',
 'com',
 'score',
 'baixo',
 'até',
 'entenderia',
 'app',
 'da',
 'digo',
 'é',
 'uma',
 'bosta',
 'mil',
 'vezes',
 'nubank@newspotgabs',
 '@nubank',
 'so',
 'queria',
 'que',
 'me',
 'dessem',
 'limite@vitorswu',
 '@nubank',
 '@bradesco',
 'nossa',
 'direto',
 'os',
 'caras',
 'aumentam',
 'meu',
 'limite',
 'kk',
 'e',
 'olha',
 'q',
 'eu',
 'nem',
 'trabalhochegou',
 'o',
 'meu',
 'cartão',
 'da',
 'nubank',
 'aaaaa',
 '😍😍até',
 'hj',
 'o',
 'novo',
 'cartão',
 'não',
 'chegou',
 'quando',
 'fui',
 'perguntar',
 'a',
 'atendente',
 'da',
 '@nubank',
 'disse',
 'q',
 'não',
 'tinha',
 'nada',
 'disso',
 'registrado',
 'e',
 'q',
 'eu',
 'tinha',
 'cancelado',
 'o',
 'cartão',
 'pq',
 'não',
 'queria',
 'mais',
 '@malualcure1',
 '@nubank',
 'taxa',
 '6',
 '50',
 'no',
 'saque',
 'pra',
 'quebrar',
 'a',
 'cara',
 'e',
 'por',
 'a',
 'frase',
 'em',
 'efeito

In [187]:
totdas_palavras_irrelevantes = texto_IR_clean.split()
totdas_palavras_irrelevantes

['@rodrigo_rhr',
 '@nubank',
 '@bradesco',
 'aff',
 'pistola',
 'estou',
 'kkk',
 'se',
 'eu',
 'estivesse',
 'devendo',
 'ou',
 'com',
 'score',
 'baixo',
 'até',
 'entenderia',
 'app',
 'da',
 'digo',
 'é',
 'uma',
 'bosta',
 'mil',
 'vezes',
 'nubank@newspotgabs',
 '@nubank',
 'so',
 'queria',
 'que',
 'me',
 'dessem',
 'limite@vitorswu',
 '@nubank',
 '@bradesco',
 'nossa',
 'direto',
 'os',
 'caras',
 'aumentam',
 'meu',
 'limite',
 'kk',
 'e',
 'olha',
 'q',
 'eu',
 'nem',
 'trabalhochegou',
 'o',
 'meu',
 'cartão',
 'da',
 'nubank',
 'aaaaa',
 '😍😍até',
 'hj',
 'o',
 'novo',
 'cartão',
 'não',
 'chegou',
 'quando',
 'fui',
 'perguntar',
 'a',
 'atendente',
 'da',
 '@nubank',
 'disse',
 'q',
 'não',
 'tinha',
 'nada',
 'disso',
 'registrado',
 'e',
 'q',
 'eu',
 'tinha',
 'cancelado',
 'o',
 'cartão',
 'pq',
 'não',
 'queria',
 'mais',
 '@malualcure1',
 '@nubank',
 'taxa',
 '6',
 '50',
 'no',
 'saque',
 'pra',
 'quebrar',
 'a',
 'cara',
 'e',
 'por',
 'a',
 'frase',
 'em',
 'efeito

In [188]:
serie_relevantes = pd.Series(totdas_palavras_relevantes)
serie_relevantes

0      @rodrigo_rhr
1           @nubank
2         @bradesco
3               aff
4           pistola
           ...     
888           custa
889              me
890         aceitar
891              no
892         crédito
Length: 893, dtype: object

In [189]:
serie_irrelevantes = pd.Series(totdas_palavras_irrelevantes)
serie_irrelevantes

0      @rodrigo_rhr
1           @nubank
2         @bradesco
3               aff
4           pistola
           ...     
888           custa
889              me
890         aceitar
891              no
892         crédito
Length: 893, dtype: object

In [190]:
tabela_relevantes  = serie_relevantes.value_counts()
tabela_relevantes

o                 28
de                26
meu               26
@nubank           24
e                 23
                  ..
kjkkkkkk           1
nesse              1
🤦🏻‍♂️@kernzera     1
@malualcure1       1
algum              1
Length: 422, dtype: int64

In [191]:
tabela_relevantes_relativa = serie_relevantes.value_counts(True)
tabela_relevantes_relativa

o                 0.031355
de                0.029115
meu               0.029115
@nubank           0.026876
e                 0.025756
                    ...   
kjkkkkkk          0.001120
nesse             0.001120
🤦🏻‍♂️@kernzera    0.001120
@malualcure1      0.001120
algum             0.001120
Length: 422, dtype: float64

In [192]:
tabela_irrelevantes_relativa = serie_irrelevantes.value_counts(True)
tabela_irrelevantes_relativa

o                 0.031355
de                0.029115
meu               0.029115
@nubank           0.026876
e                 0.025756
                    ...   
kjkkkkkk          0.001120
nesse             0.001120
🤦🏻‍♂️@kernzera    0.001120
@malualcure1      0.001120
algum             0.001120
Length: 422, dtype: float64

In [193]:
serie_Total = pd.Series(texto_total.split())
tabela_Total_relativa = serie_Total.value_counts(True)
tabela_Total_relativa

o          0.031355
meu        0.029115
de         0.029115
@nubank    0.026876
e          0.025756
             ...   
lute       0.001120
podia      0.001120
vocês      0.001120
6          0.001120
mesma      0.001120
Length: 422, dtype: float64

In [207]:
#verificando a que texto o tweet se refere:
def relevancia(frase):
    frase = cleanup(frase.lower())
    P_R_dado_TT = tabela_relevantes_relativa[frase.split()].prod()
    P_IR_dadoTT = tabela_irrelevantes_relativa[frase.split()].prod()
    if P_R_dado_TT>P_IR_dadoTT:
        return'Relevante'
    else:
        return'Irrelevante'

In [223]:
#Teste:
lista = []
for i in range(len(dados.Treinamento)):
    if relevancia(dados.Treinamento[i]) == "Relevante":
        lista.append("relevante")
    elif relevancia(dados.Treinamento[i]) == "Irrelevante":
        lista.append("irrelevante")

KeyError: "None of [Index(['@zeaugustobraz', 'unica', 'sensação', 'possível'], dtype='object')] are in the [index]"

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**